In [ ]:
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI, RateLimitError
from tqdm import tqdm

import backoff
import json
import os
import random
import re
import pandas as pd

In [ ]:
data_path = "C:\\Users\\nickm\\Documents\\stanford\\cs234\\market_stats.xlsx"

df = pd.read_excel(data_path, usecols="A:D")

In [ ]:
client = OpenAI(api_key="PLACEHOLDER")
# model = "o3-mini-2025-01-31"
model = "gpt-4o"


@backoff.on_exception(backoff.expo, RateLimitError)
def get_completion(prompt: str, units: str):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": f"You are an expert assistant at estimating market size, TAM, and other relevant statistics. Please box your final answer as just a single number without any units, e.g. \\boxed{{1.0}}. We will interpret your answer as being in {units}.\n" + prompt}],
        temperature=1.0,
    )
    return response.choices[0].message.content

In [ ]:
re.match(r"[\s\S]*\\boxed\{(.*?)\}.*", "tidsdfsdfksdfsdf ooga booga \\boxed{7345345345} sdhfksdfsdkf").group(1)

In [ ]:
def grade_estimate(answer: str, reference: float):
    assert isinstance(reference, float)
    match = re.match(r"[\s\S]*\\boxed\{(.*?)\}.*", answer)
    if match is not None:
        raw_answer = match.group(1)
        try:
            extracted_answer = float(re.sub(r"[^\d\.]", "", raw_answer))
        except:
            print(f"Could not convert {raw_answer} to float. Defaulting to reward=0")
            return (0, answer, reference)
        return (int((abs(extracted_answer - reference) / reference) <= 0.05), answer, reference)
    else:
        return (0, answer, reference)

In [ ]:
def process_example(row):
    answer = get_completion(row[0], row[1])
    return grade_estimate(answer=answer.strip(), reference=float(row[2]))

In [ ]:
with ThreadPoolExecutor() as executor:
    grades = list(tqdm(executor.map(process_example, df.values)))

In [ ]:
# --- NEW DATA ---
# o1-mini: 0.021011673151750974
# o3-mini: 0.029571984435797664
# gpt-4o: 0.031906614785992216

# NEW EVAL ON NEW DATA
# o3-mini: 0.33870270657692936
# gpt-4o: 0.423577507871906

In [ ]:
# --- GRADER BUG ---
# o1-mini: 0.26381322957198444, 0.2723735408560311 
#    (October, 2023)
# o1-preview: 0.38443579766536967 
#    (October, 2023)
# o3-mini: 
#    (October, 2023)
# gpt-4o-2024-08-06: 0.3120622568093385 
#    (October, 2023)

In [ ]:
fname = f"./{model}-base-results.jsonl"
if not os.path.exists(fname):
    with open(fname, "w") as f:
        for g in grades:
            ex = {"reference": g[2], "answer": g[1], "grade": g[0]}
            f.write(json.dumps(ex) + '\n')

In [ ]:
# import json
# with open("o3-mini-2025-01-31-base-results.jsonl", "r") as f:
#     examples = [json.loads(line) for line in f]

In [ ]:
def score(x):
    # Approx. equal to 1 at |x|=0
    # Approx. equal to 0 at |x|=1
    # Assign higher reward as we get closer to |x|=0
    # R(0.5) = 0.04, R(0.25) = 0.46, R(0.1) = 0.88, ...
    # return 1 / np.sqrt(2 * np.pi * 0.1592) * np.exp(-0.5 * (x / 0.2) ** 2)
    return 1 / np.sqrt(2 * np.pi * 0.1592) * np.exp(-0.5 * (x / 0.3) ** 2)


def smoother_accuracy(examples):
  rewards = []
  for example in examples:
    completion = example['answer']
    reference = example['reference']
    match_obj = re.match(r"[\s\S]*\\boxed{(.*?)}.*", completion)
    if match_obj is not None:
      raw_answer = match_obj.group(1)
      try:
        extracted_answer = float(re.sub(r"[^\d\.]", "", raw_answer))
        rewards.append(
            2 * score(float(abs(extracted_answer - float(reference))) / float(reference))
        )
      except Exception as e:
        print(e)
        rewards.append(0.0)
    else:
        rewards.append(0.0)
  return rewards

In [ ]:
examples[4]

In [ ]:
abs((122200000000.0 - 1300000000000)) / 122200000000.0

In [ ]:
rewards = smoother_accuracy(examples)
sum(rewards) / len(rewards)

In [ ]:
i = 1
answer = get_completion(df.values[i][0], df.values[i][1])
print(answer)

# Generate market sizing SFT data

In [ ]:
topics = [
    "Groceries",
    "Rent or mortgage",
    "Utilities (electricity, water, gas)",
    "Internet and cable",
    "Cell phone bills",
    "Transportation (gas, car payments, public transit)",
    "Health insurance",
    "Car insurance",
    "Life insurance",
    "Home insurance",
    "Clothing",
    "Shoes",
    "Electronics (phones, computers, TVs)",
    "Furniture",
    "Appliances",
    "Home cleaning supplies",
    "Toiletries (soap, toothpaste, shampoo)",
    "Beauty products (makeup, skincare)",
    "Gym memberships",
    "Fitness equipment",
    "Streaming services (Netflix, Spotify, etc.)",
    "Dining out (restaurants, cafes, takeout)",
    "Coffee and tea",
    "Snacks",
    "Alcohol",
    "Pet supplies (food, toys, grooming)",
    "Childcare or babysitting",
    "School supplies",
    "Tuition fees",
    "Student loans",
    "Medical expenses (doctor visits, medications)",
    "Prescription glasses or contacts",
    "Vitamins and supplements",
    "Personal care (haircuts, manicures)",
    "Gifts for holidays and birthdays",
    "Travel and vacations",
    "Airfare",
    "Hotel stays",
    "Luggage and travel accessories",
    "Event tickets (concerts, sports, theater)",
    "Hobbies (art supplies, books, crafts)",
    "Music lessons or classes",
    "Educational courses or workshops",
    "Home repairs",
    "Lawn care and gardening",
    "Pool maintenance",
    "Pest control",
    "Taxes (income, property, etc.)",
    "Credit card payments",
    "Loans (personal, car, home)",
    "Subscriptions (magazines, newsletters, etc.)",
    "Auto maintenance (oil changes, repairs)",
    "Gasoline",
    "Car washes",
    "Online shopping (clothes, gadgets, etc.)",
    "Apps and games",
    "Cloud storage services",
    "Charitable donations",
    "Memberships (clubs, associations, etc.)",
    "Insurance deductibles",
    "Bank fees (account maintenance, ATM fees)",
    "Storage unit rentals",
    "Home security systems",
    "Landscaping services",
    "Snow removal services",
    "Wedding expenses",
    "Baby items (diapers, formula)",
    "Furniture moving",
    "Coffee subscriptions",
    "Pool supplies",
    "Hair care products (shampoo, conditioner)",
    "Cleaning services",
    "Cooking utensils and kitchen tools",
    "Outdoor equipment (bikes, camping gear)",
    "Car rental",
    "Laundry (detergent, services)",
    "Craft supplies",
    "Photography equipment",
    "Professional services (lawyers, accountants)",
    "Business expenses (if self-employed)",
    "Legal fees",
    "Personal assistant or concierge services",
    "Nail care products",
    "Travel insurance",
    "Online courses or certifications",
    "Fitness classes (yoga, pilates)",
    "Tickets for theme parks or amusement parks",
    "Child's extracurricular activities (sports, dance)",
    "College application fees",
    "Home decoration items",
    "Rental equipment (for events, parties)",
    "Pool toys and inflatables",
    "Personal gifts (jewelry, watches)",
    "Car accessories (seat covers, floor mats)",
    "Winter gear (coats, gloves, boots)",
    "Sports gear (balls, shoes, clothing)",
    "Books (physical or e-books)",
    "Art or music supplies",
    "Office supplies (pens, paper, printer ink)",
    "Makeup or beauty services (facials, massages)",
    "Cryptocurrency investments",
    "Virtual gifts (in apps or social media)",
    "Custom-made clothing or shoes",
    "Antique or collectible items (coins, stamps)",
    "Exotic pets (like reptiles or birds)",
    "Plant care (fertilizers, pots)",
    "Ice cream trucks or food trucks",
    "Home workout subscriptions (Peloton, etc.)",
    "Tattoo or body piercing",
    "Massage chairs or spa devices",
    "Feng Shui consultants",
    "Aromatherapy oils and diffusers",
    "Essential oils",
    "Customized home decor (art, wall murals)",
    "Meal prep services or subscriptions",
    "Specialty coffees (from coffee roasters)",
    "Custom furniture",
    "Drone photography or videography services",
    "Designer handbags",
    "Luxury watches",
    "Fashion rentals (designer clothing for events)",
    "Private chef services",
    "Thrift store shopping",
    "High-end grooming services (barbershops, spas)",
    "Travel upgrades (business class flights, VIP services)",
    "Private jet charters",
    "Boat rentals or yacht charters",
    "Helicopter rides or experiences",
    "Psychic or tarot readings",
    "Haunted house tours or paranormal experiences",
    "Craft beer or wine tasting experiences",
    "VR (virtual reality) headsets or experiences",
    "Organic or specialty food items (kombucha, keto foods)",
    "Custom art (painting, digital artwork)",
    "Unique phone accessories (custom cases, skins)",
    "Meditation or mindfulness apps",
    "Exotic or rare teas",
    "3D printing services or equipment",
    "Fashion or interior design consultations",
    "Fireworks displays for events or holidays",
    "DIY home renovation materials",
    "High-tech kitchen gadgets (smart ovens, blenders)",
    "Pre-made frozen meals (for convenience)",
    "Temporary tattoos or henna designs",
    "Self-defense classes or weapons (pepper spray, tasers)",
    "Virtual reality games or experiences",
    "Social media promotions or influencers",
    "Home theater systems",
    "Energy-efficient home upgrades (solar panels, smart thermostats)",
    "Escape room experiences",
    "Laser hair removal services",
    "Drone racing or drone equipment",
    "Go-kart racing experiences",
    "Niche or rare books (first editions, collector's items)",
    "Space tourism deposits or experiences",
    "Customizable tech gadgets (personalized speakers, watches)",
    "Personalized stationery (note cards, planners)",
    "Artisanal or small-batch products (cheeses, soaps)",
    "Aerial yoga or acro yoga classes",
    "Charity events or fundraisers (auction bids, raffles)",
    "Personalized music playlists or mixes",
    "Experiential gifts (hot air balloon rides, bungee jumping)",
    "Luxury car rentals (sports cars, vintage cars)",
    "Model trains or collectibles",
    "Rare or vintage wines and spirits",
    "Home brewing kits (beer, wine, kombucha)",
    "Personalized clothing (embroidered jackets, shirts)",
    "Sandals or shoes from exotic locations (like Italy or Bali)",
    "Music festival tickets",
    "VIP nightclub or event tickets",
    "Vintage clothing or retro fashion",
    "Custom-designed sneakers",
    "Remote-controlled cars or gadgets",
    "Music production software or equipment",
    "Customization services (personalized phone cases, shoes)",
    "Vintage or retro video game consoles",
    "Reptile or exotic animal enclosures",
    "Wine or cheese club subscriptions",
    "Aerial drone photography sessions",
    "Organic beauty products or skincare routines",
    "Virtual property in video games (e.g., land, houses)",
    "Farm-to-table experiences or dining",
    "Subscription boxes for niche hobbies (knitting, fishing, etc.)",
    "Renting a private theater for movies",
    "Silent disco parties or events",
    "Soundproofing for rooms or studios",
    "Custom-made sneakers or athletic wear",
    "Language learning tools or software",
    "Flower delivery services for special occasions",
    "Personal shopping services",
    "Small-batch chocolate or confectionery",
    "Exclusive club memberships (golf, yacht clubs)",
    "Pet portraits or custom art of pets",
    "Digital artwork or NFT art",
    "Specialty cooking ingredients (truffles, saffron)",
    "Custom car wraps or paint jobs",
    "Memory-keeping services (scrapbooking, video editing)",
    "Collecting or restoring vintage cars",
    "Antique furniture restoration",
    "Wildlife safari or eco-tourism trips",
    "Personal concierge services",
    "Home automation systems (smart home devices)",
    "Customizable sneakers (design-your-own)",
    "Custom-made jewelry",
    "Beekeeping supplies",
    "Vintage musical instruments",
    "Wine barrel furniture",
    "Home composting kits",
    "Virtual fitness trainers",
    "Mushroom foraging tours",
    "Microbrewery equipment",
    "Niche coffee machines (cold brew, pour-over)",
    "Organic skincare treatments",
    "Small-batch olive oils",
    "Self-watering planters",
    "Outdoor adventure tours (rock climbing, zip-lining)",
    "Designer pet accessories (clothes, collars)",
    "Organic cotton clothing",
    "Natural hair care products",
    "Artisanal jams and spreads",
    "Personal travel guides or trip planners",
    "Vintage vinyl records",
    "Self-publishing books or e-books",
    "Specialized hiking gear (GPS, boots, backpacks)",
    "Pilates reformers",
    "Custom surfboards or water sports equipment",
    "Rooftop garden supplies",
    "Personal drone pilot lessons",
    "Caviar or luxury foods",
    "Custom pottery or ceramic art",
    "Yoga mats and accessories",
    "Dog grooming or pet spa services",
    "Virtual team-building experiences",
    "Rentable private swimming pools",
    "Personalized calendar prints",
    "Outdoor pizza ovens",
    "Historical tours (city or cultural tours)",
    "Professional home decluttering or organizing services",
    "Family or professional photoshoots",
    "Fitness wearables (smartwatches, fitness trackers)",
    "Fencing or sword fighting lessons",
    "Restoration of old paintings or antiques",
    "Antique weapon collections",
    "Collectible sneakers (limited-edition drops)",
    "Music lessons (for instruments or singing)",
    "Charity walk/run or marathon entry fees",
    "Exotic mushroom growing kits",
    "Virtual pet adoption or care services",
    "High-end binoculars for bird watching",
    "Rooftop bars or high-end dining experiences",
    "Renting a castle or luxury property for events",
    "Sailing lessons or yacht rental",
    "Building your own tiny house",
    "Geocaching supplies",
    "Virtual escape room experiences",
    "Designer bed linens or pillows",
    "Handcrafted soap collections",
    "Personal stylist services",
    "Virtual assistants or remote team assistants",
    "Renting luxury cars for photoshoots",
    "Travel packages for exclusive events (like the Monaco Grand Prix)",
    "Seasonal pumpkin patch visits",
    "Indoor skydiving experiences",
    "Baby name consultants",
    "Virtual reality escape rooms",
    "Ice skating rink rentals for private events",
    "Acupuncture or alternative medicine therapies",
    "Martial arts training gear or classes",
    "Ice sculpture artists for events",
    "High-end espresso machines",
    "A professional wardrobe overhaul (shopping spree with a stylist)",
    "Renting sports gear for vacations (skis, snowboards)",
    "Wine-tasting tours or experiences",
    "Eco-friendly home products (solar-powered chargers, bamboo toothbrushes)",
    "Digital marketing campaigns (Google ads, Facebook ads)",
    "Virtual real estate (like in online games or platforms)",
    "Antique clock collection or restoration",
    "Handcrafted furniture or reclaimed wood items",
    "Personalized travel mugs or tumblers",
    "Retro arcade games for home entertainment",
    "Sea glass collecting tours or workshops",
    "Fashion design or sewing lessons",
    "Custom-made piñatas for parties",
    "Historic artifact purchases (vintage documents, rare items)",
    "Renting a firetruck for a party",
    "Life-coaching sessions",
    "Renovating a vintage camper or RV",
    "Joining a secret or speakeasy club",
    "Mushroom cultivation kits for urban farming",
    "Psychic or medium consultations",
    "Rent-a-friend services (companionship for events or social gatherings)",
    "Custom graffiti art for home or office spaces",
    "Hiring a personal chef for dinner parties",
    "Renting out a movie theater for personal screenings",
    "Setting up a wine cellar",
    "Exotic pet insurance",
    "Organizing destination weddings",
    "Renting a horse or equestrian experiences",
    "Buying rare or collectible toys",
    "Creating a personal documentary or family film",
    "Ghost tours or paranormal experiences",
    "Renting out an ice cream truck for an event",
    "Skydiving lessons or tandem jumps",
    "Drone light shows for events",
    "Floating or sensory deprivation tanks",
    "Renting a vintage car for a photoshoot",
    "Underwater photography equipment",
    "Remote-controlled boats or helicopters",
    "Custom cake or pastry creations for events",
    "Becoming a member of a private club (e.g., country clubs, cigar clubs)",
    "Organizing a personal scavenger hunt",
    "Space planning services for offices or homes",
    "Renting a zoo or animal encounter for events",
    "Botanical garden memberships",
    "Custom 3D-printed items or prototypes",
    "Water park passes or memberships",
    "Extreme sports experiences (bungee jumping, paragliding)",
    "Private jet charter for short trips",
    "Renting a hot air balloon",
    "Renting a luxury RV for road trips",
    "Handcrafted leather goods (bags, wallets, belts)",
    "VIP concert experiences (backstage passes, meet-and-greets)",
    "Snowmobile rentals",
    "Food art (decorative cakes or gourmet creations)",
    "Indoor rock climbing memberships",
    "Private cooking lessons or food workshops",
    "Custom comic books or graphic novels",
    "Professional home theater installation",
    "Renting a boat for a day on the water",
    "Subscription box for gourmet snacks",
    "Renting a private beach for a day",
    "Exotic plant rental for events",
    "Video game console rentals for parties",
    "Tarot card deck collection",
    "Genealogy research services or ancestry kits",
    "High-end camping gear (luxury tents, air mattresses)",
    "Animal-assisted therapy or emotional support animal training",
    "Renting a private island for a weekend",
    "Custom-made pillows or throws",
    "Handcrafted musical instruments",
    "Whale watching or wildlife expeditions",
    "Collecting rare or vintage bicycles",
    "Experience gifts like helicopter tours or race car driving",
    "Temporary housing for relocation or vacation",
    "Buying limited-edition sneakers from designer collaborations",
    "Antique furniture auctions",
    "Off-the-grid living setups (tiny homes, yurts)",
    "Personalized fitness plans (meal plans, workout routines)",
    "Wildlife rescue donations or adoptions",
    "Time capsule creation for special events",
    "Rent-a-farm experiences",
    "Personal voice-over services (recording for projects)",
    "Renting professional sound systems for events",
    "Record-keeping services (for hobbies, sports statistics)",
    "Specialty items from international marketplaces",
    "Luxury house-sitting or caretaking services",
    "Personalized astrology charts",
    "Collectible action figures or figurines",
    "Spa day experiences (facials, massages)",
    "Rentable wine or whiskey tasting rooms",
    "Bird-watching equipment (binoculars, guides)",
    "High-end pet boarding or daycare",
    "Specialty cooking classes (sushi, pasta making)",
    "Custom silk or cashmere scarves",
    "Musical concert sound systems for hire",
    "Renting a movie projector for outdoor movies",
    "High-end ice cream or gelato makers",
    "Large-scale photography prints for walls",
    "Digital art commissions",
    "Fleece-lined or luxury outerwear",
    "Specialized electronic repair services",
    "Self-defense gear or classes",
    "Geode cracking kits for kids or collectors",
    "Travel photoshoots (photographers for vacation memories)",
    "Booking an entire restaurant for a private event",
    "Luxury bedding (high thread count sheets)",
    "Rare perfumes or fragrances",
    "Floating hotel stays (like on boats)",
    "Professional handwriting services",
    "High-end drinkware (crystal glasses, decanters)",
    "Custom skateboards or longboards",
    "Collectible comic books",
    "Signature cigars or cigar accessories",
    "Custom engraved jewelry",
    "Ski lessons or packages",
    "Collecting rare plants (succulents, orchids)",
    "Exotic cooking experiences (live fire grilling, underground dinners)",
    "Rent-a-limo for special events",
    "Personal or family tree artwork",
    "Experiential subscriptions (like monthly adventure boxes)",
    "Accessory customization for tech products (skins, sleeves)",
    "Biodegradable or sustainable products",
    "Unique kitchen gadgets (avocado slicers, spiralizers)",
    "Personal financial coaching",
    "High-tech bathroom devices (smart mirrors, bidets)",
    "Art gallery memberships or private viewings",
    "Outdoor gear rental (tents, kayaks, camping tools)",
    "Zip-line experiences or tours",
    "Renting luxury furniture for events",
    "Antique toy collections",
]

@backoff.on_exception(backoff.expo, RateLimitError)
def get_market_sizing_prompts(thing: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": f"Please write 50 sample market sizing questions related to '{thing}' - consider varying by demographic, geography, niche market segment, etc. to get to 20. Output format should be '1. <text>\n2. <text>\n etc.', and include no other prose or commentary."}],
        temperature=1.0,
    )
    return response.choices[0].message.content

In [ ]:
len(topics)

In [ ]:
# with ThreadPoolExecutor(max_workers=20) as executor:
#     market_sizing_responses = list(tqdm(executor.map(get_market_sizing_prompts, topics)))

In [ ]:
# qbank = set()
# for market_sizing_str in market_sizing_responses:
#     for q in re.findall("\s*\d+[.]\s*(.*)", market_sizing_str):
#         qbank.add(q)

In [ ]:
# with open("synthesized_market_sizing_prompts.txt", "w") as f:
#     for q in qbank:
#         f.write(q + '\n')

In [ ]:
with open("synthesized_market_sizing_prompts.txt", "r") as f:
    qlist = [line for line in f]

In [ ]:
@backoff.on_exception(backoff.expo, 
                      RateLimitError,
                      max_tries=3)
def get_market_sizing_response(prompt: str):
    try:
        response = client.chat.completions.create(
            model="o3-mini-2025-01-31",
            # messages=[{"role": "user", "content": f"Please summarize your reasoning in <think></think> tags and final answer in <answer></answer> tags. Include no other metadata/information: {prompt}"}],
            messages=[{"role": "user", "content": "You are an expert assistant at estimating market size, TAM, and other relevant statistics. Please enclose your verbose calculations in one set of <think></think> tags and your final answer in <answer></answer> tags and provide it just a single number without any units, e.g. <answer>1.0</answer>. We will interpret your answer as being in {units}.\n" + prompt}],
            temperature=1.0,
        )
        return response.choices[0].message.content
    except:
        return None

random.seed(712)
random.shuffle(qlist)

# with ThreadPoolExecutor(max_workers=7) as executor:
#     market_sizing_cot_traces = list(tqdm(executor.map(get_market_sizing_response, qlist[5000:10000])))

In [ ]:
with open("synthesized_market_sizing_cot_traces.jsonl", "a", encoding="utf-8") as f:
    for trace in market_sizing_cot_traces:
        f.write(json.dumps({"trace": trace}) + '\n')

In [ ]:
market_sizing_cot_traces[305]

In [ ]:
with open("synthesized_market_sizing_cot_traces.jsonl", "r", encoding="utf-8") as f:
    msct = [json.loads(line)['trace'] for line in f]

msct == market_sizing_cot_traces

In [ ]:
len(market_sizing_cot_traces)

In [ ]:
import json
import random

with open("synthesized_market_sizing_cot_traces.jsonl", "r", encoding="utf-8") as f:
    msct = [json.loads(line)['trace'] for line in f]

# with open("synthesized_market_sizing_prompts.txt", "r") as f:
#     qlist = [line for line in f]

# random.seed(712)
# random.shuffle(qlist)

# with open("synthesized_market_sizing_sft_data.jsonl", "w") as f:
#     for idx, cot in enumerate(msct):
#         if cot:
#             example = {"messages": [
#                 {"role": "user", "content": qlist[idx]},
#                 {"role": "assistant", "content": cot},
#             ]}
#             f.write(json.dumps(example) + '\n')

In [ ]:
with open("synthesized_market_sizing_sft_data.jsonl", "a") as f:
    for idx in range(5000):
        if msct[idx]:
            example = {"messages": [
                {"role": "user", "content": qlist[idx]},
                {"role": "assistant", "content": msct[idx]},
            ]}
            f.write(json.dumps(example) + '\n')

In [ ]:
[idx for idx, i in enumerate(qlist) if 'e-commerce' in i and 'consumer rating' in i]

In [ ]:
random.Random(712).shuffle(qlist)
qlist[0]

In [ ]:
qlist[10], msct[10]

In [ ]:
with open("synthesized_market_sizing_prompts_v2.txt", "a") as f:
    for i in range(5000, 10000):
        f.write(qlist[i])

In [ ]:
with open("synthesized_market_sizing_prompts_v2.txt", "r") as f:
    qlist = f.readlines()

In [ ]:
qlist[5000], msct[5000]

In [ ]:
# Notes for later
# some issue with the shuffling led to inconsistent indexing between traces and questions :I
# restart notebook and shuffle qlist on 712 TWICE to get qlist[:5000]
# restart notebook and shuffle qlist on 712 ONCE to get qlist[5000:10000]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def score(x):
    # Approx. equal to 1 at |x|=0
    # Approx. equal to 0 at |x|=1
    # Assign higher reward as we get closer to |x|=0
    # R(0.5) = 0.04, R(0.25) = 0.46, R(0.1) = 0.88, ...
    # return 1 / np.sqrt(2 * np.pi * 0.1592) * np.exp(-0.5 * (x / 0.2) ** 2)
    return 2 / np.sqrt(2 * np.pi * 0.1592) * np.exp(-0.5 * (x / 0.3) ** 2)

plt.plot([i / 100 for i in list(range(100))], [score(i / 100) for i in range(100)],
         label=r"$2e^{-\frac{1}{2}(\frac{x}{0.3})^2}$")
plt.xlabel("Relative Error")
plt.ylabel("Reward")
plt.legend(prop={"size": 20})
plt.title("Market Sizing Estimate Accuracy Reward Fn")
# plt.savefig("2smooth0.3_acc_fn.png")